In [136]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #data visalisation
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import svm



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [153]:
#Loading data
train_data = pd.read_csv('../input/titanic/train.csv')
test_data  = pd.read_csv('../input/titanic/test.csv')

train_data = train_data.drop('Name',axis = 1).drop('Ticket',axis = 1).drop('Cabin',axis = 1).dropna()
train_data = pd.get_dummies(data= train_data,prefix = 'Sex',columns = ['Sex'],drop_first = True)
train_data = pd.get_dummies(data= train_data,prefix = 'Embarked',columns = ['Embarked'],drop_first = True)

x = np.hstack((train_data.Pclass.to_numpy().reshape(-1,1),train_data.Embarked_S.to_numpy().reshape(-1,1),train_data.Embarked_Q.to_numpy().reshape(-1,1),train_data.SibSp.to_numpy().reshape(-1,1),train_data.Age.to_numpy().reshape(-1,1),train_data.Sex_male.to_numpy().reshape(-1,1),train_data.Fare.to_numpy().reshape(-1,1),train_data.Parch.to_numpy().reshape(-1,1)))

y = train_data.Survived.to_numpy()

test_data = test_data.drop('Name',axis = 1).drop('Ticket',axis = 1).drop('Cabin',axis = 1).fillna(0)
test_data = pd.get_dummies(data= test_data,prefix = 'Sex',columns = ['Sex'],drop_first = True)
test_data = pd.get_dummies(data= test_data,prefix = 'Embarked',columns = ['Embarked'],drop_first = True)
x_test = np.hstack((test_data.Pclass.to_numpy().reshape(-1,1),test_data.Embarked_S.to_numpy().reshape(-1,1),test_data.Embarked_Q.to_numpy().reshape(-1,1),test_data.SibSp.to_numpy().reshape(-1,1),test_data.Age.to_numpy().reshape(-1,1),test_data.Sex_male.to_numpy().reshape(-1,1),test_data.Fare.to_numpy().reshape(-1,1),test_data.Parch.to_numpy().reshape(-1,1)))
p_id = np.hstack((test_data.PassengerId.to_numpy().reshape(-1,1)))


In [159]:
# Linear Regression Model
lr_model = LinearRegression(normalize = True)
lr_model.fit(x,y)
h = lr_model.predict(x)
print(lr_model.score(x,y))
lr_model.coef_

0.4010431168905343


array([-1.87433328e-01, -6.63698938e-02, -9.87199353e-02, -5.08083853e-02,
       -6.42230855e-03, -4.85383294e-01,  1.96053730e-04, -1.06914269e-02])

In [155]:
# Logistic Regression Model
lr_model1 = LogisticRegression(max_iter=500,)
lr_model1.fit(x,y)
h = lr_model1.predict(x)
print(lr_model1.score(x,y))
lr_model1.coef_

0.8033707865168539


array([[-1.16048900e+00, -3.44844645e-01, -5.73661831e-01,
        -3.47304460e-01, -4.22297609e-02, -2.49969960e+00,
         1.72784001e-03, -4.79572848e-02]])

In [156]:
#SVM classifier 
clf = svm.SVC()
clf.fit(x,y)
h = clf.predict(x)
print(clf.score(x,y))

0.6769662921348315


In [158]:
#Performing Model on Test data
h_hat = lr_model1.predict(x_test)
h_hat = h_hat.reshape(-1,1)
result = np.hstack((p_id.reshape(-1,1),h_hat))
df = pd.DataFrame(result, columns = ['PassengerId', 'Survived'])
df.to_csv('prediction_svm.csv', index=False)